# PROGETTO BIONFORMATICA DV 
------------------------------------------------------------------------------------------------------------------------------

### 0. Data Preparation

In [16]:
import pandas as pd 
import itertools 
import collections as cl
import re
import numpy as np

In [219]:
f_path="https://raw.githubusercontent.com/LorenzoMauri/Progetto-Bioinformatica/master/data/ace2.gtf"
data= pd.read_csv(f_path,sep='\t',skiprows=2,names=['X','chromosome','feature','start','end','score','strand','frame','attributes'])
data.head(3)

,X,chromosome,feature,start,end,score,strand,frame,attributes
0,X,Ensembl,gene,15464246,15556529,.,+,.,ID=ENSG00000102010.15;Name=ENSG00000102010.15;...
1,X,Ensembl,gene,15561033,15602148,.,-,.,ID=ENSG00000130234.11;Name=ENSG00000130234.11;...
2,X,Ensembl,gene,15561047,15675608,.,-,.,ID=ENSG00000285602.1;Name=ENSG00000285602.1;bi...


In [220]:
def ID (stringa):
    if re.findall('ID=(.*);',stringa) != None:
        return "".join( re.findall('ID=(.*?);',stringa))
    elif re.findall('ID=(.*)',stringa) != None:
        return "".join( re.findall('ID=(.*?)',stringa))
    else:
        return None

def nome (stringa):
    if re.findall('Name=(.*);',stringa) != None:
        return "".join( re.findall('Name=(.*?);',stringa))
    elif re.findall('Name=(.*)',stringa) != None:
        return "".join( re.findall('Name=(.*?)',stringa))
    else:
        return None
    
def biotype (stringa):
    if re.findall('biotype=(.*);',stringa) !=[]:
        return ("".join( re.findall('biotype=(.*);',stringa)))
    elif re.findall('biotype=(.*)',stringa) != []:
        return ("".join(  re.findall('biotype=(.*)',stringa)))
    else:
        return None
    
def parent (stringa):
    if re.findall('Parent=(.*);',stringa) !=[]:
        return("".join(re.findall('Parent=(.*);',stringa)))
    elif re.findall('Parent=(.*)',stringa) !=[]:
        return("".join(re.findall('Parent=(.*)',stringa)))
    else:
        return None
    
data['ID']=data['attributes'].apply(ID)
data['nome']=data['attributes'].apply(nome)
data['biotype']=data['attributes'].apply(biotype)
data['parent']=data['attributes'].apply(parent)


In [221]:
data=data.drop(columns='attributes')


### 1 . Leggere il file in un DataFrame, rimuovendo le righe che non si riferiscono al gene (il file contiene anche un miRNA).
Chromosome X: 15561033-15602148 reverse strand (-)

In [222]:
# Gene
data.drop(data[(data['feature'] == 'gene') & (data['nome'] != 'ENSG00000130234.11')].index, inplace = True)


# Trascritti
data.drop(data[(data['feature'] == 'transcript') & (data['parent'] != 'ENSG00000130234.11')].index,inplace = True)



In [223]:
data


,X,chromosome,feature,start,end,score,strand,frame,ID,nome,biotype,parent
1,X,Ensembl,gene,15561033,15602148,.,-,.,ENSG00000130234.11,ENSG00000130234.11,protein_coding,None
30,X,Ensembl,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
31,X,Ensembl,transcript,15561036,15566287,.,-,.,ENST00000471548.5,ENST00000471548.5,processed_transcript,ENSG00000130234.11
32,X,Ensembl,transcript,15561880,15566709,.,-,.,ENST00000473851.1,ENST00000473851.1,processed_transcript,ENSG00000130234.11
33,X,Ensembl,transcript,15594887,15602148,.,-,.,ENST00000484756.1,ENST00000484756.1,processed_transcript,ENSG00000130234.11
...,...,...,...,...,...,...,...,...,...,...,...,...
1519,X,Ensembl,exon,15664718,15664805,.,-,.,,ENSESTE00000059618,None,ENSESTT00000015656.1
1520,X,Ensembl,exon,15664337,15664395,.,-,.,,ENSESTE00000059619,None,ENSESTT00000015656.1
1521,X,Ensembl,exon,15659016,15659101,.,-,.,,ENSESTE00000059621,None,ENSESTT00000015656.1
1522,X,Ensembl,exon,15644916,15645029,.,-,.,,ENSESTE00000059609,None,ENSESTT00000015656.1


In [225]:
data_punto1=data[(data['start']>=15561033)&(data['end']<=15602148)]
data_punto1.head(10)

,X,chromosome,feature,start,end,score,strand,frame,ID,nome,biotype,parent
1,X,Ensembl,gene,15561033,15602148,.,-,.,ENSG00000130234.11,ENSG00000130234.11,protein_coding,None
30,X,Ensembl,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
31,X,Ensembl,transcript,15561036,15566287,.,-,.,ENST00000471548.5,ENST00000471548.5,processed_transcript,ENSG00000130234.11
32,X,Ensembl,transcript,15561880,15566709,.,-,.,ENST00000473851.1,ENST00000473851.1,processed_transcript,ENSG00000130234.11
33,X,Ensembl,transcript,15594887,15602148,.,-,.,ENST00000484756.1,ENST00000484756.1,processed_transcript,ENSG00000130234.11
34,X,Ensembl,transcript,15561033,15602069,.,-,.,ENST00000427411.1,ENST00000427411.1,protein_coding,ENSG00000130234.11
136,X,Ensembl,intron,15595004,15600725,.,-,.,,intron00065,None,ENST00000252519.8
137,X,Ensembl,intron,15592323,15594844,.,-,.,,intron00066,None,ENST00000252519.8
138,X,Ensembl,intron,15591857,15592228,.,-,.,,intron00067,None,ENST00000252519.8
139,X,Ensembl,intron,15589457,15591712,.,-,.,,intron00068,None,ENST00000252519.8


### 2. Calcolare il numero di esoni per ogni trascritto

esoni con stesso parent e trascritti con stesso id 

In [226]:
df_esoni = data_punto1[(data_punto1['feature']=='exon')]
df_trascritti = data_punto1[(data_punto1['feature']=='transcript')]
data_punto2=pd.merge(df_esoni,df_trascritti,how='inner',left_on='parent',right_on='ID')
data_punto2.shape

(47, 24)

In [231]:
data_punto2[data_punto2['parent_x']==data_punto2['ID_y']].groupby(['ID_y']).size()

ID_y
ENST00000252519.8    18
ENST00000427411.1    19
ENST00000471548.5     4
ENST00000473851.1     3
ENST00000484756.1     3
dtype: int64

### 3. Calcolare la lunghezza di ogni trascritto

In [242]:
data_punto3=data_punto1.copy()
data_punto3.columns

Index(['X', 'chromosome', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'ID', 'nome', 'biotype', 'parent', 'lunghezza_trascritti'],
      dtype='object')

In [243]:
data_punto3['lunghezza_trascritti']= data_punto3['end']-data_punto3['start']
data_punto3[data_punto3['feature']=='transcript']

C:\Users\loren\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,X,chromosome,feature,start,end,score,strand,frame,ID,nome,biotype,parent,lunghezza_trascritti
30,X,Ensembl,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11,39927
31,X,Ensembl,transcript,15561036,15566287,.,-,.,ENST00000471548.5,ENST00000471548.5,processed_transcript,ENSG00000130234.11,5251
32,X,Ensembl,transcript,15561880,15566709,.,-,.,ENST00000473851.1,ENST00000473851.1,processed_transcript,ENSG00000130234.11,4829
33,X,Ensembl,transcript,15594887,15602148,.,-,.,ENST00000484756.1,ENST00000484756.1,processed_transcript,ENSG00000130234.11,7261
34,X,Ensembl,transcript,15561033,15602069,.,-,.,ENST00000427411.1,ENST00000427411.1,protein_coding,ENSG00000130234.11,41036


### 4. Calcolare le proteine associate a tali trascritti

In [25]:
data_punto4=data_punto1.copy()
data_punto3[(data_punto3['gene_y']=='transcript')&(data_punto3['biotype_y']=='protein_coding')][['gene_y','biotype_y']]

,gene_y,biotype_y
0,transcript,protein_coding
1,transcript,protein_coding
2,transcript,protein_coding
3,transcript,protein_coding
4,transcript,protein_coding
...,...,...
165,transcript,protein_coding
166,transcript,protein_coding
167,transcript,protein_coding
168,transcript,protein_coding


### 5. Determinare le regioni genomiche (esone o porzione di esone) che sono presenti in tutti i trascritti. Ogni regione genomica è identificata da una posizione di inizio e una di fine.

In [69]:
data_punto5=data_punto2
#data_punto5[(data_punto5['start_y']<=data_punto5['start_x'])&(data_punto5['end_y']>=data_punto5['end_x'])]


In [70]:
data_punto5

,chromosome_x,gene_x,start_x,end_x,score_x,strand_x,frame_x,ID_x,nome_x,biotype_x,...,gene_y,start_y,end_y,score_y,strand_y,frame_y,ID_y,nome_y,biotype_y,parent_y
0,Ensembl,exon,15600726,15600960,.,-,.,,ENSE00003897519,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
1,Ensembl,exon,15594845,15595003,.,-,.,,ENSE00000894057,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
2,Ensembl,exon,15592229,15592322,.,-,.,,ENSE00000894058,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
3,Ensembl,exon,15591713,15591856,.,-,.,,ENSE00000894059,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
4,Ensembl,exon,15589344,15589456,.,-,.,,ENSE00000894060,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Ensembl,exon,15571624,15571796,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1
200,Ensembl,exon,15570295,15570353,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1
201,Ensembl,exon,15567726,15567826,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1
202,Ensembl,exon,15566253,15566369,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1


In [71]:
data_punto5['start_y'].min()


15561033

In [72]:
data_punto5['end_y'].max()

15602148

In [76]:
data_punto5[(data_punto5['start_x']>=data_punto5['start_y'].min())&(data_punto5['end_x']<=data_punto5['end_y'].max())]

,chromosome_x,gene_x,start_x,end_x,score_x,strand_x,frame_x,ID_x,nome_x,biotype_x,...,gene_y,start_y,end_y,score_y,strand_y,frame_y,ID_y,nome_y,biotype_y,parent_y
0,Ensembl,exon,15600726,15600960,.,-,.,,ENSE00003897519,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
1,Ensembl,exon,15594845,15595003,.,-,.,,ENSE00000894057,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
2,Ensembl,exon,15592229,15592322,.,-,.,,ENSE00000894058,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
3,Ensembl,exon,15591713,15591856,.,-,.,,ENSE00000894059,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
4,Ensembl,exon,15589344,15589456,.,-,.,,ENSE00000894060,None,...,transcript,15561033,15600960,.,-,.,ENST00000252519.8,ENST00000252519.8,protein_coding,ENSG00000130234.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Ensembl,exon,15571624,15571796,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1
200,Ensembl,exon,15570295,15570353,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1
201,Ensembl,exon,15567726,15567826,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1
202,Ensembl,exon,15566253,15566369,.,-,.,,,None,...,transcript,15564024,15600960,.,-,.,compmerge.1687.Testes.chrX.1,compmerge.1687.Testes.chrX.1,aligned_transcript,compmerge.1687.Testes.chrX.1


In [166]:
data_punto3

NameError: name 'data_punto3' is not defined